In [21]:
import glob
from typing import DefaultDict
import pandas as pd
import json
import sys
import numpy as np

In [204]:
# Opening JSON file
f = open('data/category_domains.json')
category_domains_json = json.load(f)
category_domains_dict = json.loads(category_domains_json)

In [205]:
# Opening JSON file
f = open('data/tracker_category.json')
tracker_category_json = json.load(f)
tracker_category_dict = json.loads(tracker_category_json)

In [206]:
# Opening JSON file
f = open('data/tracker_owner.json')
tracker_owner_json = json.load(f)
tracker_owner_dict = json.loads(tracker_owner_json)

In [207]:
# Opening JSON file
f = open('data/successed_hostnames.txt')
successed_hostnames_arr = json.load(f)

In [208]:
# Opening JSON file
f = open('data/site_rank.txt')
site_rank_dict = json.load(f)

In [209]:
api_calls = pd.read_csv("data/100k_nyc_api_calls.csv")

In [210]:
api_calls.head()

,site_domain,initial_url,final_url,script_url,description,access_type,arguments,frameUrl,return_value,script_domain,tracker_categories,tracker_owner,is_tracker,hostname
0,lele247.top,https://www.pecellele247.me/,https://lele247.top/,https://files.sitestatic.net/assets/jquery/jqu...,Navigator.userAgent,get,NaN,NaN,NaN,sitestatic.net,NaN,NaN,0,www.pecellele247.me
1,lele247.top,https://www.pecellele247.me/,https://lele247.top/,https://files.sitestatic.net/assets/fancybox/j...,Navigator.userAgent,get,NaN,NaN,NaN,sitestatic.net,NaN,NaN,0,www.pecellele247.me
2,lele247.top,https://www.pecellele247.me/,https://lele247.top/,https://lele247.top/js/ugsports/app-desktop.js...,Navigator.userAgent,get,NaN,NaN,NaN,lele247.top,NaN,NaN,0,www.pecellele247.me
3,lele247.top,https://www.pecellele247.me/,https://lele247.top/,https://cdn.livechatinc.com/tracking.js,Navigator.userAgent,get,NaN,NaN,NaN,livechatinc.com,Embedded Content,LiveChat,0,www.pecellele247.me
4,lele247.top,https://www.pecellele247.me/,https://lele247.top/,https://cdn.livechatinc.com/tracking.js,Navigator.userAgent,get,NaN,NaN,NaN,livechatinc.com,Embedded Content,LiveChat,0,www.pecellele247.me


In [27]:
fp_attempts = pd.read_csv("data/100k_nyc_fp_attempts.csv")

In [28]:
fp_attempts.head()

,script_url,fp_type,initial_url,final_url,site_domain,script_domain,is_third_party,sc_accessed_ua,frame_url,tracker_categories,tracker_owner,is_tracker,hostname
0,https://js.ad-score.com/x.html?v=1846275&pid=1...,canvas,http://www.panorama.com.al/,http://www.panorama.com.al/,panorama.com.al,ad-score.com,True,False,False,NaN,Adscore,1,www.panorama.com.al
1,https://c.adsco.re/,webrtc,https://asiangay.tv/,https://asiangay.tv/,asiangay.tv,adsco.re,True,True,True,Ad Motivated Tracking Ad Fraud Analytics Audie...,Adscore,1,asiangay.tv
2,https://c.adsco.re/,canvas,https://asiangay.tv/,https://asiangay.tv/,asiangay.tv,adsco.re,True,True,True,Ad Motivated Tracking Ad Fraud Analytics Audie...,Adscore,1,asiangay.tv
3,https://www.molottery.com/r-ill-What-went-figh...,canvas_font,https://www.molottery.com/,https://www.molottery.com/,molottery.com,molottery.com,False,False,False,NaN,NaN,0,www.molottery.com
4,https://www.molottery.com/r-ill-What-went-figh...,canvas,https://www.molottery.com/,https://www.molottery.com/,molottery.com,molottery.com,False,False,False,NaN,NaN,0,www.molottery.com


# Crawl Statistics:

In [29]:
TOTAL_NUM_SUCC_SITES = 89763

# Sites where high entropy API was called

In [30]:
high_entropy_api_calls = api_calls[(api_calls.description=='NavigatorUAData.getHighEntropyValues')]

In [31]:
high_entropy_api_calls.hostname.nunique()

53148

# Take a look at the third parties calling high entropy API

In [32]:
third_party_high_entropy_api_calls = high_entropy_api_calls[(high_entropy_api_calls.site_domain!=high_entropy_api_calls.script_domain)]

# Number of sites where high entropy API was called by a third party:

In [33]:
third_party_high_entropy_api_calls.hostname.nunique()

52392

In [34]:
print('PERCENTAGE: ', third_party_high_entropy_api_calls.hostname.nunique()/TOTAL_NUM_SUCC_SITES)

PERCENTAGE:  0.5836703318739347


# These domains are the most popular third party domains calling this high entropy api

In [35]:
third_party_high_entropy_api_calls.drop_duplicates(['hostname']).script_domain.value_counts().head(20)

googletagmanager.com     32594
googlesyndication.com     7209
doubleclick.net           4401
googletagservices.com     1650
googleadservices.com       753
quantserve.com             733
sharethis.com              453
taboola.com                382
clarity.ms                 230
gemius.pl                  198
gstatic.com                192
statcounter.com            162
wpadmngr.com               159
tynt.com                   150
godpvqnszo.com             123
adsco.re                   123
sc-static.net              111
adocean.pl                  81
heapanalytics.com           81
pubmatic.com                73
Name: script_domain, dtype: int64

In [36]:
distinct_third_party_calls = third_party_high_entropy_api_calls.drop_duplicates(['hostname', 'script_url'])

In [37]:
categories_num_dict = defaultdict(int)
for i,req_domain in distinct_third_party_calls.script_domain.items():
    for category, domains in category_domains_dict.items():
        if req_domain in  domains:
            categories_num_dict[category]+=1

In [38]:
sorted(categories_num_dict.items(), key=lambda x:x[1], reverse=True)

[('Advertising', 93915),
 ('Ad Motivated Tracking', 92671),
 ('Audience Measurement', 67486),
 ('Third-Party Analytics Marketing', 61241),
 ('Analytics', 60443),
 ('Action Pixels', 18405),
 ('Embedded Content', 7514),
 ('CDN', 6499),
 ('Social - Share', 2576),
 ('Ad Fraud', 2132),
 ('Session Replay', 186),
 ('Badge', 141),
 ('Online Payment', 13),
 ('SSO', 13),
 ('Social - Comment', 1),
 ('Social Network', 1)]

In [39]:
len(distinct_third_party_calls)

114489

# Now we will only consider calls by a tracker domains:

In [78]:
tracker_high_entropy_api_calls = third_party_high_entropy_api_calls[third_party_high_entropy_api_calls.is_tracker==1]

# Number of sites where high entropy API was called by a tracker domain:

In [82]:
tracker_high_entropy_api_calls.hostname.nunique()

51630

In [83]:
print('PERCENTAGE: ', tracker_high_entropy_api_calls.hostname.nunique()/TOTAL_NUM_SUCC_SITES)

PERCENTAGE:  0.5751813107850673


# Most popular tracker domains calling high entropy API:

In [84]:
tracker_high_entropy_api_calls.drop_duplicates(['site_domain']).script_domain.value_counts().head(20)

googletagmanager.com     28929
googlesyndication.com     6843
doubleclick.net           3633
googletagservices.com     1414
googleadservices.com       673
quantserve.com             437
taboola.com                330
clarity.ms                 192
statcounter.com            161
wpadmngr.com               152
adsco.re                   121
godpvqnszo.com             117
gemius.pl                  113
tynt.com                   107
sc-static.net               96
adocean.pl                  74
hhbypdoecp.com              55
heapanalytics.com           51
browser-update.org          47
pubmatic.com                44
Name: script_domain, dtype: int64

# Trackers' Categories:

In [85]:
distinct_reqs = tracker_high_entropy_api_calls.drop_duplicates(['hostname','script_url'])

In [86]:
categories_num_dict = defaultdict(int)
for i,req_domain in distinct_reqs.script_domain.items():
    for category, domains in category_domains_dict.items():
        if req_domain in  domains:
            categories_num_dict[category]+=1

In [87]:
sorted(categories_num_dict.items(), key=lambda x:x[1],reverse=True)

[('Advertising', 92940),
 ('Ad Motivated Tracking', 91712),
 ('Audience Measurement', 67115),
 ('Third-Party Analytics Marketing', 60290),
 ('Analytics', 59937),
 ('Action Pixels', 18387),
 ('Embedded Content', 4934),
 ('CDN', 4644),
 ('Ad Fraud', 2132),
 ('Social - Share', 465),
 ('Session Replay', 185),
 ('Badge', 128)]

In [88]:
categories_num_dict = defaultdict(set)
for index, row in distinct_third_party_calls.iterrows():
    script_domain = row['script_domain']
    site_domain = row['site_domain']
    for category, domains in category_domains_dict.items():
        if script_domain in  domains:
            categories_num_dict[category].add(site_domain)

In [89]:
sorted_list = sorted(categories_num_dict.items(), key=lambda x:len(x[1]),reverse=True)

In [90]:
for i in sorted_list:
    print(i[0],len(i[1]))

Ad Motivated Tracking 44084
Advertising 43976
Audience Measurement 40901
Third-Party Analytics Marketing 40491
Analytics 40347
Action Pixels 13224
Embedded Content 4523
CDN 4342
Social - Share 2338
Ad Fraud 1339
Session Replay 132
Badge 124
Online Payment 13
SSO 13
Social - Comment 1
Social Network 1


In [91]:
len(tracker_high_entropy_api_calls.drop_duplicates(['hostname','script_url']))/len(high_entropy_api_calls.drop_duplicates(['hostname','script_url']))

0.9388387074637364

# Number of distinct sites where we detected FPing

In [92]:
fp_attempts.hostname.nunique()

7315

In [93]:
fp_attempts.script_url.nunique()

4689

In [94]:
third_party_fp_attempts = fp_attempts[fp_attempts.script_domain!=fp_attempts.site_domain]

In [95]:
third_party_fp_attempts.script_url.nunique()

2190

In [159]:
tracker_fp_attempts = third_party_fp_attempts[third_party_fp_attempts.is_tracker==1]

In [160]:
tracker_fp_attempts.script_url.nunique()

784

# Number of distinct sites where we detected FPing by third-party

In [161]:
third_party_fp_attempts = fp_attempts[fp_attempts.site_domain!=fp_attempts.script_domain]

In [162]:
third_party_fp_attempts.hostname.nunique()

5137

# Number of distinct sites where we detected FPing by tracker domain

In [164]:
tracker_fp_attempts = third_party_fp_attempts[third_party_fp_attempts.is_tracker==1]

In [165]:
tracker_fp_attempts.hostname.nunique()

2685

# Let's take a look at only FP attempts(also using high entropy API):

In [166]:
fp_attempts_with_high_entropy_api = fp_attempts[fp_attempts.sc_accessed_ua]

In [167]:
fp_attempts_with_high_entropy_api.script_url.nunique()

545

In [168]:
third_party_fp_attempts_with_high_entropy_api = fp_attempts[(fp_attempts.script_domain!=fp_attempts.site_domain)&(fp_attempts.sc_accessed_ua)]

In [169]:
third_party_fp_attempts_with_high_entropy_api.script_url.nunique()

274

In [172]:
tracker_fp_attempts_with_high_entropy_api = third_party_fp_attempts_with_high_entropy_api[third_party_fp_attempts_with_high_entropy_api.is_tracker==1]

In [173]:
tracker_fp_attempts_with_high_entropy_api.script_url.nunique()

157

# These are distinct sites where high entopy API was called and FPing collected:

In [174]:
len(fp_attempts_with_high_entropy_api.drop_duplicates('hostname'))

1337

# These are distinct sites where high entopy API was called and FPing collected by third party:

In [175]:
third_party_fp_at_with_high_entropy_api = fp_attempts_with_high_entropy_api[fp_attempts_with_high_entropy_api.script_domain!=fp_attempts_with_high_entropy_api.site_domain]

In [176]:
len(third_party_fp_at_with_high_entropy_api.drop_duplicates('hostname'))

1043

In [177]:
third_party_fp_at_with_high_entropy_api.drop_duplicates('hostname').hostname.head(20)

1                      asiangay.tv
14     www.malayalamsexstories.one
26                   movieplay.sbs
28                       omaha.com
44             www.hdfilmlenta.com
47                     xnxxcom.xyz
49                      xvedio.fun
51            privatehomeclips.com
53                        x-xx.pro
61       www.telugusexstories.blog
94                   xn--c1aem.org
113                   siapcoli.xyz
124              www.xxxmovies.pro
143                aflamincest.com
174                     vietsex.me
177                    masahub.net
181                animalzoosex.me
200                    zoosuck.com
223                 filmeporno.xxx
230                   hornyxxx.win
Name: hostname, dtype: object

# These domains are the most popular third party domains calling this high entropy api and also attempting FPing

In [178]:
third_party_fp_at_with_high_entropy_api.script_domain.value_counts().head(20)

adsco.re            580
wpadmngr.com        520
fpnpmcdn.net        147
fab34e9d51.com      106
shopeemobile.com     95
9f50508b3c.com       92
yabidos.com          76
bad027ae34.com       50
aibsgc.com           48
zatnoh.com           30
cdn-net.com          28
xdiwbc.com           28
genieegroup.com      23
traincdn.com         20
akamaihd.net         18
dnofd.com            14
srvmath.com          14
perimeterx.net       14
hmstats.com          13
seondf.com           12
Name: script_domain, dtype: int64

# These are distinct sites where high entopy API was called and FPing collected by tracker domain:

In [179]:
tracker_fp_at_with_high_entropy_api = third_party_fp_at_with_high_entropy_api[third_party_fp_at_with_high_entropy_api.is_tracker==1]

In [180]:
len(tracker_fp_at_with_high_entropy_api.drop_duplicates('hostname'))

815

In [181]:
tracker_fp_at_with_high_entropy_api.script_domain.value_counts().head(20)

adsco.re              580
wpadmngr.com          520
fab34e9d51.com        106
9f50508b3c.com         92
yabidos.com            76
bad027ae34.com         50
zatnoh.com             30
akamaihd.net           18
srvmath.com            14
perimeterx.net         14
hmstats.com            13
fpcdn.io               12
ocdn.eu                12
cdn-net.com            12
anura.io               12
ad-score.com           11
yqmxfz.com             10
a3595ccd38.com         10
rubiconproject.com      6
acexedge.com            4
Name: script_domain, dtype: int64

# Category of the tracker domains calling high entopy API

In [211]:
for domain in tracker_fp_at_with_high_entropy_api.script_domain.value_counts().keys():
    if domain in tracker_category_dict:
        print('DOMAIN: ', domain, "\nCategories: ", tracker_category_dict[domain])
    if domain in tracker_owner_dict:
        print("Owner: ", tracker_owner_dict[domain])

DOMAIN:  adsco.re 
Categories:  Ad Motivated Tracking Ad Fraud Analytics Audience Measurement
Owner:  Adscore
Owner:  Human Security
DOMAIN:  perimeterx.net 
Categories:  Ad Motivated Tracking Ad Fraud Analytics Audience Measurement
Owner:  PerimeterX
Owner:  Ringier Axel Springer
Owner:  Adscore
DOMAIN:  rubiconproject.com 
Categories:  Ad Motivated Tracking Advertising
Owner:  Magnite
Owner:  Human Security
DOMAIN:  pubmatic.com 
Categories:  Ad Motivated Tracking Advertising Ad Fraud Analytics Audience Measurement Third-Party Analytics Marketing
Owner:  PubMatic
DOMAIN:  3lift.com 
Categories:  Ad Motivated Tracking Advertising
Owner:  TripleLift
DOMAIN:  moatads.com 
Categories:  Ad Motivated Tracking Advertising Ad Fraud Analytics Audience Measurement Third-Party Analytics Marketing Action Pixels
Owner:  Oracle
DOMAIN:  smartadserver.com 
Categories:  Ad Motivated Tracking Advertising
Owner:  Smartadserver
Owner:  Dealer Dot Com
DOMAIN:  microsoft.com 
Categories:  Ad Motivated Tr

# Which ua information was accessed most by tracker scripts?

In [427]:
def get_call_counts_by_arg():
    high_entropy_agrs_call_counts = defaultdict(set) 
    high_entropy_agrs_call_counts_script_domain = defaultdict(set) 
    high_entropy_agrs_call_counts_tuple = defaultdict(set) 
    for index, row in third_party_high_entropy_api_calls.drop_duplicates(['hostname', 'script_url']).iterrows():
        initial_url = row['initial_url']
        script_domain = row['script_domain']
        script_url = row['script_url']
        if row['description'] == 'NavigatorUAData.getHighEntropyValues':
            if '0' in row['arguments']:
                arg_str = (row['arguments'].replace("'", '"'))
                if 'None' in arg_str:
                    continue
                json_args = json.loads(arg_str)
                for arg in json_args['0']:
#                     if arg == None:
#                         print(initial_url)
                    high_entropy_agrs_call_counts[arg].add(initial_url)
                    high_entropy_agrs_call_counts_script_domain[arg].add(script_domain)
                    high_entropy_agrs_call_counts_tuple[arg].add(script_url+initial_url)
    return high_entropy_agrs_call_counts, high_entropy_agrs_call_counts_script_domain,high_entropy_agrs_call_counts_tuple

In [428]:
high_entropy_agrs_call_counts,high_entropy_agrs_call_counts_script_domain,high_entropy_agrs_call_counts_tuple = get_call_counts_by_arg()

In [429]:
high_entropy_agrs_call_counts_script_domain

defaultdict(set,
            {'platform': {'123multimedia.com',
              '143flix.com',
              '14i8trbbx4.com',
              '2158novffp.com',
              '268stephe5en3king.com',
              '2cnjuh34jbpoint.com',
              '2mdn.net',
              '2qj7mq3w4uxe.com',
              '361lu.com',
              '3lift.com',
              '425.degree',
              '4hfchest5kdnfnut.com',
              '5toft8or7on8tt.com',
              '79xmz3lmss.com',
              '864feb57ruary.com',
              '9f50508b3c.com',
              'a3595ccd38.com',
              'abdurantom.com',
              'abkoxlikbzs.com',
              'acacdn.com',
              'acdcdn.com',
              'acexedge.com',
              'achcdn.com',
              'acscdn.com',
              'actualno.com',
              'ad-drop.jp',
              'ad-mv.de',
              'ad-spire.net',
              'ad-stir.com',
              'adbrothers.nl',
              'ademon.net',
           

In [430]:
for key, value in high_entropy_agrs_call_counts.items():
    print(key, len(value))

platform 51529
platformVersion 52214
architecture 51229
model 52270
uaFullVersion 50743
bitness 50874
fullVersionList 51321
wow64 50132
mobile 7615
brands 5208
uaFulVersion 7


In [431]:
sorted_list = sorted(high_entropy_agrs_call_counts.items(), key=lambda x:len(x[1]),reverse=True)

In [432]:
for i in sorted_list:
    print(i[0],len(i[1]))

model 52270
platformVersion 52214
platform 51529
fullVersionList 51321
architecture 51229
bitness 50874
uaFullVersion 50743
wow64 50132
mobile 7615
brands 5208
uaFulVersion 7


In [433]:
for key, value in high_entropy_agrs_call_counts_script_domain.items():
    print(key, len(value))

platform 746
platformVersion 1052
architecture 618
model 970
uaFullVersion 359
bitness 554
fullVersionList 792
wow64 80
mobile 283
brands 462
uaFulVersion 1


In [434]:
for key, value in high_entropy_agrs_call_counts_tuple.items():
    print(key, len(value))

platform 92921
platformVersion 108505
architecture 93070
model 109992
uaFullVersion 83702
bitness 86266
fullVersionList 97575
wow64 76152
mobile 11764
brands 7329
uaFulVersion 7


# How does popularity affect?

In [435]:
third_party_high_entropy_api_calls['site_rank'] = third_party_high_entropy_api_calls.apply(lambda x: site_rank_dict[x['initial_url'][:-1]], axis=1)

/tmp/ipykernel_3351568/3964068474.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  third_party_high_entropy_api_calls['site_rank'] = third_party_high_entropy_api_calls.apply(lambda x: site_rank_dict[x['initial_url'][:-1]], axis=1)


In [436]:
fp_attempts['site_rank'] = fp_attempts.apply(lambda x: site_rank_dict[x['initial_url'][:-1]], axis=1)

In [437]:
fp_attempts.drop_duplicates('initial_url').site_rank.value_counts()

100000    3232
50000     3073
10000      467
5000       411
1000       132
Name: site_rank, dtype: int64

In [438]:
api_calls['site_rank'] = api_calls.apply(lambda x: site_rank_dict[x['initial_url'][:-1]], axis=1)

In [439]:
successed_hostnames_set = set(successed_hostnames_arr)

In [440]:
api_calls[(api_calls.site_rank==1000)&(api_calls.hostname.isin(successed_hostnames_set))].initial_url.nunique()

815

In [441]:
third_party_high_entropy_api_calls.drop_duplicates('initial_url').site_rank.value_counts()

100000    26092
50000     21092
10000      2671
5000       2029
1000        508
Name: site_rank, dtype: int64